In [1]:
# Cell 1 - Setup & Load Data GNN
import pandas as pd
import numpy as np
import xgboost as xgb
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# Path file hasil output GNN (Cell 12 notebook sebelumnya)
INPUT_CSV = "../gnn_model/output/gnn_hybrid_predicted_nodes.csv"

print(f"📂 Memuat data dari: {INPUT_CSV}")
nodes = pd.read_csv(INPUT_CSV)

# Pastikan kolom target aman (isi 0 jika kosong)
nodes['is_fraud'] = nodes['is_fraud'].fillna(0).astype(int)

print(f"✅ Data berhasil dimuat. Total baris: {len(nodes)}")
nodes.head()

📂 Memuat data dari: ../gnn_model/output/gnn_hybrid_predicted_nodes.csv
✅ Data berhasil dimuat. Total baris: 1879


,node_id,mapped_id,fraud_score,is_fraud,ratio_tarif,delta_tarif,lama_rawat,degree,pagerank,community_train_fraud_rate,...,n2v_54,n2v_55,n2v_56,n2v_57,n2v_58,n2v_59,n2v_60,n2v_61,n2v_62,n2v_63
0,0,0,0.0231,0,0.000000,0.0,0.0,4,0.640479,0.0,...,0.256532,0.435936,0.067036,-0.328814,-0.439293,-0.134036,0.239671,0.145506,-0.468373,0.106788
1,1,1,0.0000,0,1.047769,150131.0,3.0,5,0.724720,0.0,...,0.180500,0.076481,-0.058738,-0.544431,-0.264163,-0.148306,0.030371,-0.045088,-0.251053,0.177910
2,2,2,0.0000,0,0.000000,0.0,0.0,62,8.130346,0.0,...,0.377480,0.069126,0.186254,-0.366588,-0.316488,-0.347047,-0.302116,0.141787,-0.155829,0.792509
3,3,3,0.0000,0,0.000000,0.0,0.0,95,12.453211,0.0,...,0.010238,-0.565841,-0.211583,-0.441610,0.033722,0.130060,-0.048193,-0.512530,0.181944,-0.280712
4,4,4,0.0000,0,0.000000,0.0,0.0,562,73.233768,0.0,...,0.211797,-0.076313,0.172875,-0.315923,0.228542,-0.063241,-0.139795,-0.067248,0.150290,-0.032489


In [2]:
# Cell 2 - Define Features
# Kita akan menggunakan semua kolom KECUALI identitas dan label sebagai fitur
ignore_cols = ['node_id', 'mapped_id', 'is_fraud', 'train_mask', 'test_mask', 'fraud_reason', 'ai_explanation', 'final_risk_score']

# Ambil semua kolom numerik + fraud_score dari GNN
# fraud_score dari GNN adalah "Golden Feature" untuk XGBoost!
feature_cols = [c for c in nodes.columns if c not in ignore_cols]

print(f"🚀 Fitur yang akan masuk ke XGBoost ({len(feature_cols)} fitur):")
print(feature_cols[:10], "...") # Tampilkan 10 pertama aja biar gak penuh

🚀 Fitur yang akan masuk ke XGBoost (74 fitur):
['fraud_score', 'ratio_tarif', 'delta_tarif', 'lama_rawat', 'degree', 'pagerank', 'community_train_fraud_rate', 'community_train_count', 'provider_train_fraud_rate', 'provider_train_count'] ...
